In [ ]:
# STEP 1: Mount Drive (if using Google Drive)
from google.colab import drive
drive.mount('/content/drive')

# STEP 2: Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import os

# STEP 3: Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# STEP 4: Set Paths and Hyperparameters
data_dir = "/content/drive/MyDrive/dataset"
BATCH_SIZE = 16
IMG_SIZE = 128
EPOCHS = 50
SEED = 42
torch.manual_seed(SEED)

# STEP 5: Data Transforms
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# STEP 6: Datasets and Dataloaders
full_dataset = datasets.ImageFolder(data_dir)
class_names = full_dataset.classes
print("Classes:", class_names)

# Split manually into 80-20
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

train_dataset.dataset.transform = train_transform
val_dataset.dataset.transform = val_transform

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# STEP 7: Define CNN Model
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * (IMG_SIZE // 8) * (IMG_SIZE // 8), 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x

model = CNNModel(num_classes=len(class_names)).to(device)

# STEP 8: Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# STEP 9: Training Loop with Best Model Saving
best_acc = 0.0
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total

    # Validation
    model.eval()
    val_correct, val_total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_acc = val_correct / val_total
    print(f"Epoch {epoch+1}/{EPOCHS} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

    # Save best model
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "/content/drive/MyDrive/best_animal_model.pth")
        print("✅ Best model saved")

# STEP 10: Evaluation
model.load_state_dict(torch.load("/content/drive/MyDrive/best_animal_model.pth"))
model.eval()

y_true = []
y_pred = []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        y_pred.extend(preds.cpu().numpy())
        y_true.extend(labels.numpy())

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using device: cuda
Classes: ['Bear', 'Bird', 'Cat', 'Cow', 'Deer', 'Dog', 'Dolphin', 'Elephant', 'Giraffe', 'Horse', 'Kangaroo', 'Lion', 'Panda', 'Tiger', 'Zebra']
Epoch 1/50 | Train Acc: 0.1055 | Val Acc: 0.1568
✅ Best model saved
Epoch 2/50 | Train Acc: 0.1601 | Val Acc: 0.1928
✅ Best model saved
Epoch 3/50 | Train Acc: 0.2039 | Val Acc: 0.2108
✅ Best model saved
Epoch 4/50 | Train Acc: 0.2360 | Val Acc: 0.2545
✅ Best model saved
Epoch 5/50 | Train Acc: 0.2920 | Val Acc: 0.3111
✅ Best model saved
Epoch 6/50 | Train Acc: 0.3659 | Val Acc: 0.3753
✅ Best model saved
Epoch 7/50 | Train Acc: 0.4469 | Val Acc: 0.4036
✅ Best model saved
Epoch 8/50 | Train Acc: 0.5029 | Val Acc: 0.4473
✅ Best model saved
Epoch 9/50 | Train Acc: 0.5762 | Val Acc: 0.5013
✅ Best model saved
Epoch 10/50 | Train Acc: 0.6373 | Val Acc: 0.5501
✅ Best model saved
Epoch 11/50 | Train Acc: 0